# Machine learning - sprint 1
authors: Allart Ewoud, Van Hees Maxime, Breda Bram

#### GET AN INTIAL FEEL OF THE DATA? MOET DIT ER NOG BIJ OF NIET? --> LABO 1

## Importing data

In [ ]:
# Import pandas and read csv using pandas
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import plotly.express as px

from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

In [ ]:


original_df = pd.read_csv("tripadvisor_dataset/restaurant_listings.csv")

# display the data and see how it formulated
pd.set_option("display.max_columns", None)
original_df.head()

In [ ]:
original_df_reviews = pd.read_csv("tripadvisor_dataset/reviews.csv")
original_df_reviews.head()

## Split in test & train data

The goal of machine learning is to build models on train data that are able to make predictions on unseen test data. So we first try to split our data into a trainset and a test set before we move on to the other steps.

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(original_df, random_state=0, train_size = 0.8)
df_train_reviews, df_test_reviews = train_test_split(original_df_reviews, random_state=0, train_size = 0.8)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df = df_train.select_dtypes(include=numerics)



## Data clean-up

First we start with cleaning of the less complicated data. This includes the collumns general rating, number of reviews and rank.

### General rating, number of reviews & rank

General rating komt in een tekstuele vorm, uit deze vorm moet enkel het eerste getal worden gehaald. We merken ook op dat het een aantal -1 values heeft, deze veranderen we door NaN. 



In [ ]:
# general rating
df["general rating"] = df_train["general rating"].apply(lambda x: float(str(x).split(' ')[0]))
df["general rating"] = pd.to_numeric(df["general rating"])
df["general rating"] = df["general rating"].replace(-1, np.nan)

# number of reviews
df["number of reviews"] = df_train["number of reviews"].apply(lambda x: int(str(x).split(' ')[0]))
df["number of reviews"] = pd.to_numeric(df["number of reviews"])

# rank
df["rank"] = df_train["rank"].str.split("#").str[1]
df["rank"] = pd.to_numeric(df["rank"])

df.head()

### Restaurants with no reviews

2 options:

- we remove the restaurants that have no reviews, and thus nu general rating. But will this impact the model we will train later on? 

```df2 = df.loc[(df['number of reviews'] == 0) & (df['general rating'] == -1.0)]```

- we don't remove them and assume the model will learn by itself that these entries aren't that useful.

Kga dit eens vragen aan iemand die data science als werk doet.

### Price range
A collumn that is a bit more complicated is price range, this collumn will be split in 2, after this the average is taken.

In [ ]:
upper_bound = df_train["price range"].str.split(" - ").str[1]
upper_bound = upper_bound.str.split('€').str[1]
upper_bound = pd.to_numeric(upper_bound)

lower_bound = df_train["price range"].str.split(" - ").str[0]
lower_bound = lower_bound.str.split('€').str[1]
lower_bound = pd.to_numeric(lower_bound)

sn.displot(upper_bound)
sn.displot(lower_bound)

In [ ]:
from sklearn import preprocessing

average = (upper_bound-lower_bound)/2

transformer = preprocessing.FunctionTransformer(pd.cut, kw_args={'bins': 10, 'labels': 
                                                                 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'retbins': True})
avg_quantfied = transformer.fit_transform(average)

#visualize the data
sn.displot(avg_quantfied[0])

df['price range'] =  pd.to_numeric(avg_quantfied[0])
df.head()

### Food, service & value rating

In [ ]:
pd.plotting.scatter_matrix(df[["food rating","service rating","value rating", "general rating"]], figsize = (8,8))

Notice in the chart above, that in food rating, service rating, value rating & atmosphere rating there are alot of -1.
- In the cases of food, value & service rating we are going to replace them with the average
- In the case of atmosphere rating, we are going to drop this column, you can see that most of the data here is -1. So in this case this looks like the best solution.

In [ ]:
mean = df["service rating"].loc[df["service rating"] != -1].mean()
df["service rating"] = df["service rating"].replace(-1,mean)

mean = df["food rating"].loc[df["food rating"] != -1].mean()
df["food rating"] = df["food rating"].replace(-1,mean)

mean = df["value rating"].loc[df["value rating"] != -1].mean()
df["value rating"] = df["value rating"].replace(-1,mean)

df = df.drop(columns=["atmosphere rating"])

pd.plotting.scatter_matrix(df[["food rating","service rating","value rating"]], figsize = (8,8))

### Meals

In [ ]:
# Getting all different types off meals
types = []
for i in df_train['meals'].fillna(value=""):
    types.extend(map(lambda x: x.strip(), i.split(",")))
types = set(types)


# For every type we are going to create a collumn that represents the type
for i in types:
    if i != "":
        df[i] = df_train["meals"].apply(lambda x: str(x)).apply(lambda x: int(i in x))

# displays full table
#with pd.option_context('display.max_rows', None, 'display.max_columns', None): display(df)
df.head()

#### <font color='red'>VRAAG: kunnen we de opties reduceren, bv brunch & breakfast? Later hiernaar kijken</font>

### Cuisines

In [ ]:
types = []
for i in df_train['cuisines'].fillna(value=""):
    types.extend(map(lambda x: x.strip(), i.split(",")))
types = set(types)
print("Amount of different cuisines:",len(types))

# prints the amout of times a cuisine occurs in the data
# set _print on True to enable printing of frequencies
_print = True


freq = {}
for i in types:  
    if i != "":
        freq[i] = df_train["cuisines"].apply(lambda x: str(x)).apply(lambda x: int(i in x)).sum()


#print("\n")
#print("-"*20, "frequency tabel", "-"*20) 
#for key, value in freq.items():
#    print('{:<25}'.format(key ), '{:>4}'.format(value) , "times")
percentile75 = np.quantile(list(freq.values()), 0.75)
percentile50 = np.quantile(list(freq.values()), 0.5)

print("50-percentile:",percentile50)
print("75-percentile:",percentile75)


freq = { key:value for (key,value) in freq.items() if value > percentile50}

for (key,value) in freq.items():
    if key != "":
        df[key] = df_train["cuisines"].apply(lambda x: int(key in str(x)))
   
#TODO add others collumn
df

The boxplot shows the distribution of the frequencies of the cuisines. Here we can see that a lot of cuisines occur only a few times. This may presents an oppertunity to reduce these to a categorie like "others".

In [ ]:
fig = plt.figure(figsize =(4, 10))
ax = fig.add_axes([0, 0, 1, 1])
bp = ax.boxplot(freq.values())
plt.show()  

### Address

The address field has the following structure in most cases:
    1. Streetname + number + ","
    2. City + corresponding postal code
    3. Country (here Belgium in all cases)
Upon closer inspection we can see that sometimes field 1. and 2. are comma-separted, sometimes not.
The number behind the streetname sometimes consists of two numbers (can be split with ' ' or '-' or '/' or ...)

In [ ]:
locator = Nominatim(user_agent="ML")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

def get_coordinates(name):
    location = locator.geocode(name)
    if location is None:
        return pd.Series([np.nan, np.nan])
    else:
        return pd.Series([location.latitude, location.longitude])
    
df[['latitude','longitude']] = df_train['address'].fillna('').apply(get_coordinates)
df

In [ ]:
fig = px.scatter_mapbox(df[['latitude', 'longitude']], lat="latitude", lon="longitude", height=500, width=700)

fig.update_layout(mapbox_style="open-street-map")
fig.show()
# https://stackoverflow.com/questions/53233228/plot-latitude-longitude-from-csv-in-python-3-6

### Restaurant features

Extract different restaurant feautures from the dataset

In [ ]:
# Create a dict that holds all restaurant feautures and how much they appear
feature_types = {}
for i in df_train['restaurant features'].fillna(value=""):
    for f in i.split(','):
        if i != "":
            if f.strip() in feature_types.keys():
                feature_types[f.strip()] += 1
            else:
                feature_types[f.strip()] = 1
           
print("Amount of features:" , len(feature_types.keys()))

# omzetten naar dataframe
df_features = pd.DataFrame(feature_types, index=[0]).transpose()
df_features.columns = ['Amount']
df_features.sort_values(['Amount'], ascending=False, inplace=True)
df_features.plot(kind='barh', figsize=(12,12))

for (key,value) in feature_types.items():
    if key != "":
        df[key] = df_train["restaurant features"].apply(lambda x: int(key in str(x)))

### Special Diets

In [ ]:
# check which special diets exist in the data
diet_types = {}
for i in df_train['special diets'].fillna(value=""):
    for f in i.split(','):
            if f.strip() in diet_types.keys():
                diet_types[f.strip()] += 1
            else:
                diet_types[f.strip()] = 1
           
print("Amount of diets:" , len(diet_types.keys()))
print(diet_types)

# check if available diet options have influence on the average rating per restaurant
diet_avg_review = {}
for d in diet_types.keys():
    if len(df_train[df_train['special diets'].str.contains(d) == True]) >=  100: #only use data of a diet type if there are more than 100 restaurants with that diet type
        diet_avg_review[d] = df[df_train['special diets'].str.contains(d) == True]['general rating'].mean()
print('Average review score per diet type',diet_avg_review)
df_diets = pd.Series(diet_avg_review).rename({'': 'No diets specified in data'})
df_diets.sort_values(ascending=False).plot(kind='barh', figsize=(10,2), title='Average review score per diet')


The graph above shows that restaurants with vegan and gluten free options have a higher average review score. 

### Price tag

Bij price de andere price metriek zagen we dat er zeer veel data ontbrak, de andere metriek heeft ook een aantal nadelen, het is namelijk zo dat 1 goedkoop gericht op 1 door gerecht in een resto de volleige metriek beinvoeld.

In [ ]:
price_options = []

for i in df_train['tags'].fillna(value=""):
    for j in i.split('|'):
        if '$' in j:
            price_options.append(j)
            
price_options = set(price_options)
price_options = sorted(price_options, key= lambda x: len(x.split('-')[0].strip()))
       
def formatPriceRange(tags):
    for i in reversed(range(0,3)):
        if price_options[i] in tags:
            return i + 1
    return nan
       
df["price tag"] = df_train['tags'].apply(lambda x: formatPriceRange(str(x)))

### Website & menu

In [ ]:
df['has website url'] = df_train['website url'].apply(lambda x: str(x) != "nan")
df['has menu url'] = df_train['menu url'].apply(lambda x: str(x) != "nan")

means = {}
means['website url'] = df.loc[df['has website url'] == 1]['general rating'].mean()
means['no website url'] = df.loc[df['has website url'] == 0]['general rating'].mean()

means['menu url'] = df.loc[df['has menu url'] == 1]['general rating'].mean()
means['no menu url'] = df.loc[df['has menu url'] == 0]['general rating'].mean()
pd.Series(means).plot(kind='barh', figsize=(10,2), title='Average score based on urls')

### michelin & travelers choice & timetable

De data is duidelijk gewoon een boolean, dit is eenvoudig om te zetten naar een int.

In [ ]:
print(df_train['michelin'].value_counts())
print(df_train['travelers choice'].value_counts())
print(df_train['timetable'].value_counts())

df["michelin"] = df_train["michelin"].astype(int)
df["travelers choice"] = df_train["travelers choice"].astype(int)
df["timetable"] = df_train["timetable"].astype(int)

### Phone number and Email address

In [ ]:
print("Telefoonnummer aanwezig?")
print(df_train['phone number'].fillna('').astype(bool).value_counts())
print("Emailadres aanwezig?")
print(df_train['email address'].fillna('').astype(bool).value_counts())

df['phone number'] = df_train['phone number'].fillna('').astype(bool).astype(int)
df['email address'] = df_train['email address'].fillna('').astype(bool).astype(int)

### Tags 

Check if there is data in the tags column that is not already in other columns (excluding the price tag, because we already covered that)

In [ ]:
# split and strip cuisines (,)
# split and strip special diets (,)
# split and strip tags (|)
columnsToAdd = []
def testRow(cuisines, specialDiets, tags):
    storedData = []
    # voeg eerst alles toe dat al is opgeslagen door de cuisines en special diets kolommen te verwerken
    for i in str(cuisines).split(','):
        if (i != 'nan'):
            storedData.append(i.strip())  
    for i in str(specialDiets).split(','):
        if (i != 'nan'):
            storedData.append(i.strip()) 
    
    # chech of alles in de tags kolom al is opgeslagen        
    for i in str(tags).split('|'):
        if('$' not in i):
            if(i not in storedData and i != 'nan'):
                columnsToAdd.append(i.strip())
                print('Fout bij tag:', i, tags)
                
    
df_train[['cuisines','special diets','tags']].apply(lambda x: testRow(*x), axis=1)

columnsToAdd = set(columnsToAdd)

print(columnsToAdd)

# er blijken nog enkele kolommen te ontbreken in onze df dataframe, er staat dus soms data in de tags kolom die nergens anders staat in de rij van een bepaald element

# de kolommen die ontbraken in df toevoegen

def addColumnValues(cuisines, specialDiets, tags, column):
    if (column in (str(tags).split('|') or str(cuisines).split(',') or str(specialDiets).split())):
        return 1
    return 0
for i in columnsToAdd:
    df[i] = df_train[['cuisines','special diets','tags']].apply(lambda x: addColumnValues(*x,i), axis=1)

### Restaurant popularity
Because there is no column that cal tell us how popular a restaurant is, we will try to estimate the popularity of a restaurant by taking the amount of reviews in consideration. We assume that if a restaurant gets a lot of reviews, alot of people go there. We also take the interval between the first and the last review into account. If we divide the amount of reviews by the interval we get a value that estimates the popularity of the restaurant. 

In [ ]:
...

## Data analysis

### michelin

- To start off, the ratings are checked to make sure there are no negative values. We do this cause negative values could influence the results.
- Subsequently we look at what michelin means in terms of ratings.

In [ ]:
ratings = {"general rating", "food rating", "service rating", "value rating"}

print('-'*10,"Amount of negative values :",'-'*10)
for i in ratings:
    print(i,df[i].loc[df[i] < 0].count())
    


avg_ratings = {}

for i in ratings:
    avg_ratings[i + ' of michelin'] = df.loc[df['michelin'] == 1][i].mean()
    avg_ratings[i + ' of no michelin'] = df.loc[df['michelin'] == 0][i].mean()
    
pd.Series(avg_ratings).plot(kind='barh', figsize=(10,2), title='average ratings based on michelin')

Notice in the chart above, that michelin restaurants on average score higher. It is here important to note that we do not conclude that a michelin restaurant will give a restaurant higher ratings. It is rather so that only very good restaurants become michelin restaurants an so logicly they recuived on average higher ratings.

### Timetable

In [ ]:
avg_ratings = {}
for i in ratings:
    avg_ratings[i + ' with timetable'] = df.loc[df['timetable'] == 1][i].mean()
    avg_ratings[i + ' with no timetable '] = df.loc[df['timetable'] == 0][i].mean()
    
pd.Series(avg_ratings).plot(kind='barh', figsize=(10,2), title='average ratings based on timetables')

Notice in the chart above, that on average restaurants with a timetable score higher. Het is belangrijk om te beseffen dat dit niet aanzien wordt als een factor die de ratings direct beinvloed. Er zijn kunnen verschillende redenen waarom restaurants die timetables gebruiken hoger scoren. Dit kan bijvoorbeeld een teken zijn van profesionaliteit en profesionaliteit kan leiden tot betere scores.

### travelers choice

In [ ]:
avg_ratings = {}
for i in ratings:
    avg_ratings[i + ' with travelers choice'] = df.loc[df['travelers choice'] == 1][i].mean()
    avg_ratings[i + ' without travelers choice '] = df.loc[df['travelers choice'] == 0][i].mean()
    
pd.Series(avg_ratings).plot(kind='barh', figsize=(10,2), title='average ratings based on travelers choice')

We zien dat restaurants die gemarkeerd zijn met travelers choice gemiddeld gezien hoger scoren.

### Restaurant features


In [ ]:
percentile25 = np.quantile(list(feature_types.values()), 0.25)
avg_ratings = {}
for i in feature_types.keys():
    tmp = df.loc[df[i]== 1]
    if (tmp.shape[0] > percentile25): # only use features that occur enough times, because if it only occurs a few times, results may be inaccurate
        avg_ratings[i] = tmp['general rating'].mean()
pd.Series(avg_ratings).sort_values().plot(kind='barh', figsize=(10,10), title='Average ratings based on restaurant features')

The graph above shows that having certain features in a restaurant can result in a higher review score on average. Note that the restaurant features near the bottom are not necessarily bad features to have, it may be that customers don't really care about it as well.  

### Ratings

In [ ]:
sn.heatmap(df[["general rating", "food rating", "service rating", "value rating"]].corr(), annot=True)

Notice in the chart above the red colering, this indicates a that food rating, value rating and service rating are highly correlated(0,7 - 0,9). At last you can notice that general rating is low(0,3 - 0,5) to moderately(0,5 - 0,7) correlated to the other ratings.

### ratings compared to price

In [ ]:
sn.heatmap(df[["general rating", "food rating", "service rating"
               , "value rating",'price range','price tag']].corr(), annot=True)

In the chart above you can notice that there isn't a significant correlation between the price and the rating of a restaurant.

In [ ]:
sn.heatmap(df[["rank", "food rating", "service rating"
               , "value rating",'general rating']].corr(), annot=True)

In [ ]:
pd.plotting.scatter_matrix(df[["rank", "food rating", "service rating"
               , "value rating",'general rating']], figsize = (8,8))

In [ ]:
df